In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np

# Annealing method for 2D Thompson atomic mode

We want to simulate a system composed of Coulomb charges in a two-dimensional trap (Thompson atomic model) by using the classical Monte Carlo method and using the annealing method to find the minimal energy configuration

The Metropolis Monte Carlo code to simulate a system consisting of $N$ Coulomb charges in a 2D harmonic trap. The potential energy is given by
$$E_{pot}=\sum_{i=1}^N{\frac12m\omega²r_i²}+\sum_{i<j}^N{\frac{q²}{\left|\mathbf r_i-\mathbf r_j\right|}}$$



In [ ]:
N = 5 # number of charges

